In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset/IMDB Dataset.csv


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [3]:
import pandas as pd
df=pd.read_csv('/kaggle/input/imdb-dataset/IMDB Dataset.csv')

In [4]:
df=df.sample(24000)

In [24]:
df.head()

,review,sentiment,tokenized_review
7469,"This is the weakest of the series, not much of...",negative,"[this, is, the, weakest, of, the, series, ,, n..."
7859,Fantastic movie! One of the best film noir mov...,positive,"[fantastic, movie, !, one, of, the, best, film..."
37418,This film is so bad and gets worse in every im...,negative,"[this, film, is, so, bad, and, gets, worse, in..."
32221,What gives Anthony Minghella the right to ruin...,negative,"[what, gives, anthony, minghella, the, right, ..."
46753,"OK, please believe me when I say that this is ...",negative,"[ok, ,, please, believe, me, when, i, say, tha..."


In [23]:
df=df.drop('tokenized_review')

KeyError: "['tokenized_review'] not found in axis"

In [25]:
df.to_csv('/kaggle/working/my_dataframe.csv', index=False)

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from torch.optim import AdamW
!pip install pytorch-lightning
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import DataLoader,TensorDataset
import pytorch_lightning as L
from torch.nn.utils.rnn import pad_sequence
import smart_open.local_file
from gensim.models import Word2Vec


import gensim.downloader as api

In [ ]:
import gensim.downloader as api

In [7]:
print("Loading GloVe embeddings...")
glove_vectors = api.load("glove-wiki-gigaword-300")
print("GloVe embeddings loaded.")

Loading GloVe embeddings...
GloVe embeddings loaded.


In [ ]:
import threading

stop_execution = threading.Event()

def long_running_task():
    while not stop_execution.is_set():
        # Your code here
        print("Loading GloVe embeddings...")
        glove_vectors = api.load("glove-wiki-gigaword-300")
        print("GloVe embeddings loaded.")
        if stop_execution.is_set():
            print("Execution stopped")
            break

threading.Thread(target=long_running_task).start()


In [ ]:
stop_execution.set()

In [8]:
def safe_tokenize(text):
    if isinstance(text, str):
        return word_tokenize(text.lower())
    else:
        return []

# Assuming df is your DataFrame with 'review' and 'sentiment' columns
df['tokenized_review'] = df['review'].fillna('').apply(safe_tokenize)
df = df[df['tokenized_review'].map(len) > 0]

vector_size = 300
max_seq_length = 200

def review_to_vector_sequence(review, embeddings, max_length):
    vectors = [embeddings[word] for word in review[:max_length] if word in embeddings]
    if len(vectors) == 0:
        return [np.zeros(vector_size)]
    return vectors

# Convert reviews to sequences of vectors using GloVe
print("Converting reviews to vector sequences...")
review_vector_sequences = df['tokenized_review'].apply(lambda x: review_to_vector_sequence(x, glove_vectors, max_seq_length))

def pad_vectors(vectors, max_length, padding_value=0):
    array = np.array(vectors)
    padded = np.zeros((max_length, vector_size), dtype=np.float32)
    length = min(array.shape[0], max_length)
    padded[:length, :] = array[:length, :]
    return torch.from_numpy(padded)

# Convert reviews to padded tensors
print("Creating padded tensors...")
X = pad_sequence([pad_vectors(seq, max_seq_length) for seq in review_vector_sequences], batch_first=True)
y = torch.tensor(df['sentiment'].map({'positive': 1, 'negative': 0}).values, dtype=torch.float32)

# Create tensor datasets
dataset = TensorDataset(X, y)

print("Data preparation complete. You can now use this dataset in your RNN model.")
print(f"Input shape: {X.shape}")  # Should be (num_samples, max_seq_length, vector_size)

Converting reviews to vector sequences...
Creating padded tensors...
Data preparation complete. You can now use this dataset in your RNN model.
Input shape: torch.Size([24000, 200, 300])


In [9]:
class RNN(L.LightningModule):
    def __init__(self):
        super().__init__()
        
        self.w1 = nn.Parameter(torch.Tensor(300, 64), requires_grad=True)
        self.w2 = nn.Parameter(torch.Tensor(64, 64), requires_grad=True)
        self.w3 = nn.Parameter(torch.Tensor(64, 1), requires_grad=True)
        self.bh = nn.Parameter(torch.Tensor(1, 64), requires_grad=True)
        self.bo = nn.Parameter(torch.Tensor(1, 1), requires_grad=True)
        self.init_weights()

    def init_weights(self):
        for param in [self.w1, self.w2, self.w3]:
            nn.init.xavier_uniform_(param)
        nn.init.zeros_(self.bh)
        nn.init.zeros_(self.bo)

    def rnn_math(self, input, o_t):
        o_t = torch.tanh(torch.matmul(input, self.w1) + torch.matmul(o_t, self.w2) + self.bh)
        return o_t

    def forward(self, input):
        batch_size, seq_len, _ = input.shape
        o_t = torch.zeros(batch_size, 64, device=self.device)
        for i in range(seq_len):
            o_t = self.rnn_math(input[:,i, :], o_t)
        return torch.sigmoid(torch.matmul(o_t, self.w3) + self.bo)

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=1e-3)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=1000, eta_min=1e-6)
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
            "monitor": "train_loss"
        }
    def training_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = nn.functional.binary_cross_entropy_with_logits(outputs, labels.unsqueeze(1).float())
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, sync_dist=True)
        return loss
    def validation_step(self, batch, batch_idx):
        inputs, labels = batch
        outputs = self(inputs)
        loss = nn.functional.binary_cross_entropy_with_logits(outputs, labels.unsqueeze(1).float())
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True, sync_dist=True)

In [10]:
from torch.utils.data import random_split

total_size = len(dataset)
train_size = int(0.8 * total_size)  # 80% for training
val_size = total_size - train_size  # 20% for validation

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
def create_dataloader(dataset, batch_size, shuffle=True):
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True,
        persistent_workers=True,
        prefetch_factor=2,
    )
batch_size = 8096 
dataloader = create_dataloader(dataset, batch_size)
val_dataloader = create_dataloader(val_dataset, batch_size, shuffle=False)

In [11]:
model=RNN()

In [12]:
class GPUUsageCallback(L.Callback):
    def on_train_batch_end(self, trainer, pl_module, outputs, batch, batch_idx):
        if batch_idx % 100 == 0:  # Print every 100 batches
            for i in range(torch.cuda.device_count()):
                print(f"GPU {i} memory usage: {torch.cuda.memory_allocated(i) / 1e9:.2f} GB")

trainer = L.Trainer(
    max_epochs=10,
    accelerator="gpu",
    devices=2,  # Use both T4 GPUs
    strategy="ddp_notebook",  # Distributed Data Parallel
    precision='16-mixed',
    gradient_clip_val=1.0,
    accumulate_grad_batches=2,
    )
trainer.fit(model, train_dataloaders=dataloader, val_dataloaders=val_dataloader)

2024-07-08 07:15:11.127840: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 07:15:11.127905: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 07:15:11.129898: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with 

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [26]:
import joblib
joblib.dump(model, 'my_model.joblib2')

['my_model.joblib2']

In [ ]:

model = joblib.load('/kaggle/input/model2/my_model.joblib2')

In [27]:
joblib.dump(trainer.checkpoint_callback.best_model_path,'chkpt2')

['chkpt2']

In [33]:

k = joblib.load('/kaggle/working/lightning_logs/version_8/checkpoints/epoch=4049-step=12130.ckpt', encoding='latin1')

TypeError: load() got an unexpected keyword argument 'encoding'

In [34]:
checkpoint = torch.load('/kaggle/working/lightning_logs/version_8/checkpoints/epoch=4049-step=12130.ckpt')
print(checkpoint.keys())

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers'])


In [35]:
torch.save(checkpoint, '/kaggle/working/saved_checkpoint.pth')

In [32]:
torch.save(model.state_dict(), 'model_state.pth')

In [31]:
k=joblib.load('/kaggle/working/lightning_logs/version_8/checkpoints/epoch=4049-step=12130.ckpt')

UnpicklingError: persistent IDs in protocol 0 must be ASCII strings

In [ ]:
path = joblib.load('/kaggle/input/chkpt2/chkpt2')

In [13]:
path_to_better_checkpoint=trainer.checkpoint_callback.best_model_path
trainer=L.Trainer(max_epochs=100)
trainer.fit(model,train_dataloaders=dataloader,ckpt_path=path_to_better_checkpoint)

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
2024-07-08 07:16:22.255847: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 07:16:22.255929: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 07:16:22.257456: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:361: The dirpath has changed from '/kaggle/working/lightning_logs/version_4/checkpoints' to '/kaggl

Training: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [18]:
for i in range(100):
    print(f"Review {i+1}: Label->{y[i]}  and predicted value:",model.forward((X[i:i+1]).clone().detach()))


loss=nn.functional.binary_cross_entropy_with_logits(model.forward(X[:20000]),y[:20000].unsqueeze(1))

print(loss.item())

Review 1: Label->0.0  and predicted value: tensor([[0.0053]], grad_fn=<SigmoidBackward0>)
Review 2: Label->1.0  and predicted value: tensor([[0.0053]], grad_fn=<SigmoidBackward0>)
Review 3: Label->0.0  and predicted value: tensor([[0.0003]], grad_fn=<SigmoidBackward0>)
Review 4: Label->0.0  and predicted value: tensor([[0.0025]], grad_fn=<SigmoidBackward0>)
Review 5: Label->0.0  and predicted value: tensor([[0.0087]], grad_fn=<SigmoidBackward0>)
Review 6: Label->1.0  and predicted value: tensor([[0.0183]], grad_fn=<SigmoidBackward0>)
Review 7: Label->1.0  and predicted value: tensor([[0.0053]], grad_fn=<SigmoidBackward0>)
Review 8: Label->1.0  and predicted value: tensor([[0.0326]], grad_fn=<SigmoidBackward0>)
Review 9: Label->0.0  and predicted value: tensor([[0.0049]], grad_fn=<SigmoidBackward0>)
Review 10: Label->0.0  and predicted value: tensor([[0.6519]], grad_fn=<SigmoidBackward0>)
Review 11: Label->1.0  and predicted value: tensor([[0.1369]], grad_fn=<SigmoidBackward0>)
Review 1

In [14]:
path_to_better_checkpoint=trainer.checkpoint_callback.best_model_path
trainer=L.Trainer(max_epochs=2000)
trainer.fit(model,train_dataloaders=dataloader,ckpt_path=path_to_better_checkpoint)

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:361: The dirpath has changed from '/kaggle/working/lightning_logs/version_5/checkpoints' to '/kaggle/working/lightning_logs/version_6/checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.


Training: |          | 0/? [00:00<?, ?it/s]

In [20]:
path_to_better_checkpoint=trainer.checkpoint_callback.best_model_path
trainer=L.Trainer(max_epochs=4050)
trainer.fit(model,train_dataloaders=dataloader,ckpt_path=path_to_better_checkpoint)

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:361: The dirpath has changed from '/kaggle/working/lightning_logs/version_7/checkpoints' to '/kaggle/working/lightning_logs/version_8/checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.


Training: |          | 0/? [00:00<?, ?it/s]

In [ ]:
for i in range(100):
    print(f"Review {i+1}: Label->{y[i]}  and predicted value:",model.forward((X[i:i+1]).clone().detach()))


loss=nn.functional.binary_cross_entropy_with_logits(model.forward(X[:20000]),y[:20000].unsqueeze(1))

print(loss.item())

In [21]:
model.eval()
with torch.no_grad():  # Disable gradient calculation
    output = model.forward(X[:24000])

for i in range(24000):
    if(output[i,0]>=0.5):
        output[i,0]=1
    else:
        output[i,0]=0
i=0
accuracy = accuracy_score(output,y[:24000].unsqueeze(1))
precision = precision_score(output,y[:24000].unsqueeze(1))
recall= recall_score(output,y[:24000].unsqueeze(1))
fl= f1_score(output,y[:24000].unsqueeze(1))

print(accuracy)
print(precision)
print(recall)
print(fl)

i=i+1
model.train()

0.58025
0.20804922667553633
0.8197903014416776
0.3318742538798249


RNN()

In [ ]:
import shutil
shutil.move( '/kaggle/output/output_file.csv','/kaggle/working/lightning_logs',)

In [ ]:
shutil.move('/kaggle/output/output_file.csv','/kaggle/input/imdb-dataset/output1.csv')

In [ ]:
import os
import json

def save_best_model_path(trainer):
    best_model_path = trainer.checkpoint_callback.best_model_path
    
    # Create a dictionary with the information
    info = {
        "best_model_path": best_model_path,
        "current_epoch": trainer.current_epoch,
        "global_step": trainer.global_step
    }
    
    # Save the information to a JSON file
    with open('/kaggle/working/best_model_info.json', 'w') as f:
        json.dump(info, f)

    print(f"Best model path saved: {best_model_path}")

save_best_model_path(trainer)

def load_best_model_path():
    try:
        with open('/kaggle/working/best_model_info.json', 'r') as f:
            info = json.load(f)
        return info['best_model_path']
    except FileNotFoundError:
        print("No saved best model path found.")
        return None

# Load the best model path
best_model_path = load_best_model_path()

if best_model_path:
    print(f"Loading model from: {best_model_path}")
    model = RNN.load_from_checkpoint(best_model_path)

path_to_better_checkpoint=best_model_path
trainer=L.Trainer(max_epochs=3002)

trainer.fit(model,train_dataloaders=dataloader,ckpt_path=path_to_better_checkpoint)

1000 epochs: 
0.6741309762001038,
0.58135
0.2209431485408462
0.7687983134223472
0.34324260726331474
3000 epochs:
0.615
0.7259
0.470766434413814
0.9508464205588415
0.6297446980953668

import torch
import pandas as pd
import numpy as np

def tensor_3d_to_dataframe(tensor):
    # Ensure the tensor is on CPU and convert to numpy
    if tensor.is_cuda:
        tensor = tensor.cpu()
    
    # Convert to numpy array
    array_3d = tensor.numpy()
    
    # Get the dimensions
    samples, seq_length, features = array_3d.shape
    
    # Reshape the 3D array to 2D
    array_2d = array_3d.reshape(samples, seq_length * features)
    
    # Create column names
    column_names = [f'feature_{i}_{j}' for i in range(seq_length) for j in range(features)]
    
    # Create DataFrame
    df = pd.DataFrame(array_2d, columns=column_names)
    
    return df

df= tensor_3d_to_dataframe(X)

print(df.shape)
print(df.head())

y = y.cpu().numpy()  # Assuming y is your labels tensor
df['label'] = y



import pandas as pd
import os
from datetime import datetime

def save_dataframe(df, base_filename, folder='/kaggle/working/'):
    # Create a timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Create the full filename with timestamp
    filename = f"{base_filename}_{timestamp}.csv"
    
    # Create the full path
    full_path = os.path.join(folder, filename)
    
    # Save the DataFrame to CSV
    df.to_csv(full_path, index=False)
    
    print(f"DataFrame saved to {full_path}")
    
    # Optionally, you can also save a small metadata file with the latest filename
    metadata = {'latest_file': filename}
    pd.DataFrame(metadata, index=[0]).to_csv(os.path.join(folder, f"{base_filename}_metadata.csv"), index=False)

save_dataframe(df, 'my_dataframe')

def load_latest_dataframe(base_filename, folder='/kaggle/working/'):
    metadata_file = os.path.join(folder, f"{base_filename}_metadata.csv")
    
    if os.path.exists(metadata_file):
        metadata = pd.read_csv(metadata_file)
        latest_file = metadata['latest_file'].iloc[0]
        full_path = os.path.join(folder, latest_file)
        
        if os.path.exists(full_path):
            df = pd.read_csv(full_path)
            print(f"Loaded DataFrame from {full_path}")
            return df
        else:
            print(f"Latest file {full_path} not found.")
    else:
        print(f"No metadata file found at {metadata_file}")
    
    return None

# Load the latest DataFrame
df = load_latest_dataframe('my_dataframe')